In [1]:
import os

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import optimizers
import keras
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint



Using TensorFlow backend.


In [2]:
IMG_PATH = "data/"
print(os.listdir(IMG_PATH))

['.ipynb_checkpoints', 'validation_set', 'training_set']


In [3]:
train_path = IMG_PATH + "training_set/"
validation_path = IMG_PATH + "validation_set/"

In [4]:
def number_of_imgs(path):
    print(path)
    for value in os.listdir(path):
        print(value, "has", len(os.listdir(path + value)), "imgs")

In [5]:
number_of_imgs(train_path)
number_of_imgs(validation_path)

data/training_set/
hemmorhage_data has 80 imgs
non_hemmorhage_data has 80 imgs
data/validation_set/
hemmorhage_data has 21 imgs
non_hemmorhage_data has 20 imgs


In [6]:
img_width, img_height = 224, 224
nb_train_samples = 160
nb_validation_samples = 41
epochs = 25
batch_size = 6
checkpoint_filepath = "checkpoint/model2_val_loss-{val_loss:.2f}.h5"

In [7]:
from keras.applications.vgg16 import VGG16
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

W0826 12:42:27.432030 139696020285184 deprecation_wrapper.py:119] From /home/h8953/tensorflow_gpu_env/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0826 12:42:27.444600 139696020285184 deprecation_wrapper.py:119] From /home/h8953/tensorflow_gpu_env/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0826 12:42:27.446919 139696020285184 deprecation_wrapper.py:119] From /home/h8953/tensorflow_gpu_env/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0826 12:42:27.467457 139696020285184 deprecation_wrapper.py:119] From /home/h8953/tensorflow_gpu_env/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf

In [8]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [9]:
#for layer in conv_base.layers:
    #layer.trainable = False

In [10]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [11]:
model = Sequential()
model.add(conv_base)
model.add(Flatten())                                    
model.add(Dense(64))                                    
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))                                     
model.add(Activation('sigmoid'))            

model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])

W0826 12:42:28.467057 139696020285184 deprecation.py:506] From /home/h8953/tensorflow_gpu_env/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0826 12:42:28.495534 139696020285184 deprecation_wrapper.py:119] From /home/h8953/tensorflow_gpu_env/lib/python3.5/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0826 12:42:28.501941 139696020285184 deprecation.py:323] From /home/h8953/tensorflow_gpu_env/lib/python3.5/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has t

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                1605696   
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total para

In [13]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    rescale=1. / 255,                   #Scaling RGB values from "0-255" to "0-1" 
    zoom_range=0.2,                 #Random zoom between 0 and 0.2
    horizontal_flip=True) #Random horizontal flip
               


test_datagen = ImageDataGenerator(rescale=1. / 255) #Scaling RGB values from "0-255" to "0-1". 


# Generates batches of image data for training

train_generator = train_datagen.flow_from_directory(
    train_path,                                       #Train data directory
    target_size=(img_width, img_height),                  #Image size
    batch_size=batch_size,
    class_mode="binary")                                  #Amount of classes, binary if 2 classes, else categorical


validation_generator = test_datagen.flow_from_directory(
    validation_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary")


# Lowering learning rate if no progress is made.

reduce_learning_rate = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=5,
    min_lr=0.00001,
    verbose=1)


# Defining checkpoint that saves the best model or model"s weights based on the value that we are monitoring.

checkpoint = ModelCheckpoint(
    checkpoint_filepath,
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    #save_weights_only=True,
    mode="min")


# Defining callbacks which includes checkpoint and reducing learning rate on plateau.

callbacks = [checkpoint, reduce_learning_rate]

# Training CNN

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,         
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=callbacks)

Found 160 images belonging to 2 classes.
Found 41 images belonging to 2 classes.
Epoch 1/25
26/26 [==============================] - 7s 287ms/step - loss: 0.8006 - acc: 0.5192 - val_loss: 0.6949 - val_acc: 0.5000

Epoch 00001: val_loss improved from inf to 0.69493, saving model to checkpoint/model2_val_loss-0.69.h5
Epoch 2/25
26/26 [==============================] - 6s 229ms/step - loss: 0.6903 - acc: 0.6090 - val_loss: 0.7125 - val_acc: 0.4571

Epoch 00002: val_loss did not improve from 0.69493
Epoch 3/25
26/26 [==============================] - 4s 158ms/step - loss: 0.6982 - acc: 0.5033 - val_loss: 0.7114 - val_acc: 0.5143

Epoch 00003: val_loss did not improve from 0.69493
Epoch 4/25
26/26 [==============================] - 4s 160ms/step - loss: 0.6987 - acc: 0.5159 - val_loss: 0.6958 - val_acc: 0.4857

Epoch 00004: val_loss did not improve from 0.69493
Epoch 5/25
26/26 [==============================] - 4s 165ms/step - loss: 0.7023 - acc: 0.4936 - val_loss: 0.7048 - val_acc: 0.4286